# Solving the radiative transfer equation for a single atmospheric layer using the discrete ordinates method in Python

by Dion Ho Jia Xu

In [1]:
import numpy as np
import scipy as sc

from math import pi
from numpy.polynomial import legendre
from scipy import fft
from scipy import integrate

# Table of Contents
* [1. List of parameters to set](#1.-List-of-parameters-to-set)
	* [1.1 Hyperparameter list (excludes phase function) (TODO)](#1.1-Hyperparameter-list-%28excludes-phase-function%29-%28TODO%29)
	* [1.2 Computational variable list](#1.2-Computational-variable-list)
	* [1.3 Legendre expansion of the phase function](#1.3-Legendre-expansion-of-the-phase-function)
		* [1.3.1 Phase functions (TODO)](#1.3.1-Phase-functions-%28TODO%29)
* [2. PyDISORT algorithm](#2.-PyDISORT-algorithm)
	* [2.1 Subroutines](#2.1-Subroutines)
	* [2.2 Main algorithm](#2.2-Main-algorithm)
	* [2.3 Breakdown and verification of PyDISORT](#2.3-Breakdown-and-verification-of-PyDISORT)
		* [2.3.1 Re-derivation of equations (6a) to (6d) in [1]](#2.3.1-Re-derivation-of-equations-%286a%29-to-%286d%29-in-[[1]])
		* [2.3.2 Re-derivation of equations (7a) and (7b) in [1]](#2.3.2-Re-derivation-of-equations-%287a%29-and-%287b%29-in-[[1]])
		* [2.3.3 Solving the system for each Fourier mode](#2.3.3-Solving-the-system-for-each-Fourier-mode)
		* [2.3.4 Constructing the general solution for each Fourier mode](#2.3.4-Constructing-the-general-solution-for-each-Fourier-mode)
			* [2.3.4.1 Verification of the general solution for one Fourier mode](#2.3.4.1-Verification-of-the-general-solution-for-one-Fourier-mode)
		* [2.3.5 The full solution and computation of flux, reflectivity, transmittivity](#2.3.5-The-full-solution-and-computation-of-flux,-reflectivity,-transmittivity)
			* [2.3.5.1 Verification of full solution](#2.3.5.1-Verification-of-full-solution)
			* [2.3.5.2 Verification of flux, reflectivity, transmittivity (TODO)](#2.3.5.2-Verification-of-flux,-reflectivity,-transmittivity-%28TODO%29)
			* [2.3.5.3 Timing PyDISORT](#2.3.5.3-Timing-PyDISORT)


We wish to solve the radiative transfer equation

\begin{align*}
\mu \frac{\partial u(\tau, \mu, \phi)}{\partial \tau} = \ u(\tau, \mu, \phi) &-\frac{\omega_0}{4 \pi} \int_{-1}^{1} \int_{0}^{2 \pi} p\left(\mu, \phi ; \mu', \phi'\right) u\left(\tau, \mu', \phi'\right) \mathrm{d} \phi' \mathrm{d} \mu' \\
&-\frac{\omega_0 I_0}{4 \pi} p\left(\mu, \phi ;-\mu_{0}, \phi_{0}\right) \exp\left(-\mu_{0}^{-1} \tau\right)
\end{align*}

with Dirichlet boundary conditions. This will be done numerically using the Discrete Ordinates Method.

============================================== Start of required user input ===================================================

# 1. List of parameters to set

## 1.1 Hyperparameter list (excludes phase function) (TODO)

The variable names generally follow those in Stamnes et. al.'s seminal 1988 paper [[1]](#cite-STWJ1988), with the equivalent variable in their DISORT code [[2]](#cite-Sta1999) in brackets.

* Optical depth (DTAUC), i.e. bottom of atmosphere as we define the top to be $\tau = 0$.

In [2]:
tau_prime = (25 / 8) * (10**-2)  # There may be ill-conditioning if tau_prime > 1

*TODO: Implement the scheme by Yamamoto et al. (1971) to address potential ill-conditioning if `tau_prime` $> 1$? See [[3]](#cite-SS1981) as a first reference*

* Single-scattering albedo (SSALB). We assume independence from $\tau$. The permissible range of values is $[0,1)$.

In [3]:
w0 = 0.2

In [4]:
assert w0 >= 0
assert w0 < 1

* Parameters for the incident collimated beam 

In [5]:
# Intensity (FBEAM)
I0 = 10 * pi
# Cosine of polar angle (UMU0)
mu0 = pi / 4
# Azimuthal angle (PHI0)
phi0 = pi / 3

 * Array of $\mu$-varying Dirichlet boundary conditions for Fourier modes (No direct equivalent in Stamnes' DISORT [[2]](#cite-Sta1999), but see variable *IBCND*). Computational variables `NLeg` and `NQuad` are specified below.

$$
u(0, -\mu_i, \phi) = b^-_i \sum_{m = 0}^{\mbox{NLeg}}\cos(m(\phi_0 - \phi)), \quad u\left(\tau', \mu_i, \phi \right) = b^+_i \sum_{m = 0}^{\mbox{NLeg}}\cos(m(\phi_0 - \phi)), \quad i = 1, \dots, \mbox{NQuad}
$$

The hemispheric flux contribution of each BC is $\pi b^\pm$.

In [6]:
# At top of atmosphere
b_neg = 0

# At bottom of atmosphere
b_pos = 0

* Whether to only compute flux values (ONLYFL). If `True`, `PyDISORT` will be much faster because we will only need to solve the $0$th Fourier mode integro-differential equation; see Sections [2.3.2](#2.3.2-Re-derivation-of-equations-%287a%29-and-%287b%29-in-[[1]]) and [2.3.5](#2.3.5-The-full-solution-and-computation-of-flux,-reflectivity,-transmittivity).

In [7]:
only_flux = False

## 1.2 Computational variable list

* Number of Legendre coefficients for the phase function (NMOM). Equal to the number of Fourier modes.

In [8]:
NLeg = 32

* Number of evaluation points for quadrature approximation of the polar or $\mu$ integral (NSTR). This parameter is also known as the number of "streams".

In [9]:
NQuad = 32

`NQuad` is required to be greater than $2$, even, and less than or equal to `NLeg`. It is generally a good idea to have `NQuad` = `NLeg`.

In [10]:
assert NQuad > 2
assert NQuad % 2 == 0
assert NQuad <= NLeg

## 1.3 Legendre expansion of the phase function

Following the method in [[1]](#cite-STWJ1988) but with slightly different notations and definitions, we expand the phase function

$$
p(\cos\gamma) \approx \sum_{\ell=0} g_\ell P_\ell(\cos\gamma), \quad g_\ell = \frac{2\ell + 1}{2}\int_{-1}^{1} p(\cos\gamma) P_\ell(\cos\gamma) \mathrm{d}\cos\gamma
$$

The angle $\gamma$ is between the incident vector $\left(\theta', \phi'\right)$ and the scattering vector $(\theta, \phi)$ such that

$$
\cos\gamma = \cos\theta'\cos\theta + \sin\theta'\sin\theta\cos\left(\phi'-\phi\right)
$$

and so by the addition theorem for spherical harmonics

$$
P_\ell(\cos\gamma) = P_\ell\left(\cos\theta'\right)P_\ell(\cos\theta) + 2\sum_{m=1}^\ell \frac{(\ell-m)!}{(\ell+m)!}P_\ell^m\left(\cos\theta'\right)P_\ell^m(\cos\theta)\cos\left(m\left(\phi'-\phi\right)\right)
$$

### 1.3.1 Phase functions (TODO)

Phase functions in Stamnes' DISORT [[2]](#cite-Sta1999):

    1) Isotropic
    2) Rayleigh
    3) Henyey-Greenstein with asymmetry factor GG
    4) Haze L as specified by Garcia/Siewert
    5) Cloud C.1 as specified by Garcia/Siewert
    6) Aerosol as specified by Kokhanovsky 
    7) Cloud as specified by Kokhanovsky

**Henyey-Greenstein phase function**

The exact definition varies between sources by a constant factor. The definition here follows [[1]](#cite-STWJ1988), though the phase function is not explicitly defined in the paper.

$$\frac{1-g^2}{\left(1+g^2-2 g \cos \gamma\right)^{3 / 2}}$$

*TODO: Implement delta-scaled HG function (follow [[4]](#cite-JWW1976) & [[5]](#cite-Wis1977)?)*

* Asymmetry factor (GG)

In [11]:
g = 0.75

* Phase function

`p_HG_nu` is equivalent to `p_HG` except that it uses $\nu = \cos\gamma$. `p_HG_muphi` is also equivalent except that it uses

$$
\cos\gamma = \cos\theta'\cos\theta + \sin\theta'\sin\theta\cos\left(\phi'-\phi\right)
$$

and $\mu = \cos\theta$, $\mu' = \cos\theta'$; see [[6]](#cite-MW1980) but note that $\omega_0$ is outside our phase function.

In [12]:
p_HG = lambda gamma: (1 - g**2) / ((1 + g**2 - 2 * g * np.cos(gamma)) ** (3 / 2))
p_HG_nu = lambda nu: (1 - g**2) / ((1 + g**2 - 2 * g * nu) ** (3 / 2))

def p_HG_muphi(mu, phi, mu_p, phi_p):
    mu, phi, mu_p, phi_p = np.atleast_1d(mu, phi, mu_p, phi_p)
    return np.squeeze(
        (1 - g**2)
        / (
            1
            + g**2
            - 2
            * g
            * np.moveaxis(
                np.outer(mu, mu_p)[:, :, None, None]
                + np.tensordot(
                    np.outer(np.sqrt(1 - mu**2), np.sqrt(1 - mu_p**2)),
                    np.cos(phi[:, None] - phi_p[None, :]),
                    axes=0,
                ),
                source=(2, 1),
                destination=(1, 2),
            )
        )
        ** (3 / 2)
    )

* Normalized array of $g_\ell$ values $\big((2\ell + 1) \times \mbox{PMOM}\big)$

In [13]:
m_arr = np.arange(NLeg)
Leg_coeffs = (2 * m_arr + 1) * g**m_arr

**Integral derivation of Legendre coefficients for verification**

This algorithm can be used to derive the Legendre coefficients for other phase functions. The algorithm can also be vectorized, but we would no longer be able to use `scipy.integrate.quad` for integration.

In [14]:
Leg_coeffs_test = np.empty(NLeg)
for ell in range(NLeg):
    integrand = lambda nu: p_HG_nu(nu) * legendre.Legendre(np.append(np.zeros(ell), 1))(nu)
    Leg_coeffs_test[ell] = ((2 * ell + 1) / 2) * integrate.quad(integrand, -1, 1)[0]

assert np.allclose(Leg_coeffs, Leg_coeffs_test)

print("Passed all tests")

Passed all tests


================================================ End of required user input ===================================================

==================================================== Start of algorithm ====================================================

# 2. PyDISORT algorithm

## 2.1 Subroutines

In [15]:
def generate_Ds(m):
    ells = np.arange(m, NLeg)
    Dm_term = Leg_coeffs[ells] * (
        sc.special.factorial(ells - m) / sc.special.factorial(ells + m)
    )

    degree_tile = np.tile(ells, (N, 1)).T
    asso_leg_term_pos = sc.special.lpmv(m, degree_tile, mu_arr_pos)
    asso_leg_term_neg = sc.special.lpmv(m, degree_tile, mu_arr_neg)
    # We use broadcasting instead of diagonal matrices for computational efficiency
    D_temp = (Dm_term)[None, :] * asso_leg_term_pos.T

    D_pos = (w0 / 2) * D_temp @ asso_leg_term_pos
    D_neg = (w0 / 2) * D_temp @ asso_leg_term_neg

    return D_pos, D_neg


def generate_Xs(m):
    if m == 0:
        prefactor = w0 * I0 / (4 * pi)
    else:
        prefactor = w0 * I0 / (2 * pi)

    ells = np.arange(m, NLeg)
    Xm_term = Leg_coeffs[ells] * (
        sc.special.factorial(ells - m) / sc.special.factorial(ells + m)
    )
    Xm_term2 = sc.special.lpmv(m, ells, -mu0)
    X_temp = prefactor * Xm_term * Xm_term2

    degree_tile = np.tile(ells, (N, 1)).T
    X_pos = X_temp @ sc.special.lpmv(m, degree_tile, mu_arr_pos)
    X_neg = X_temp @ sc.special.lpmv(m, degree_tile, mu_arr_neg)

    return X_pos, X_neg

## 2.2 Main algorithm

In [16]:
def PyDISORT(
    b_pos, b_neg, only_flux, NQuad, tau_prime, w0, Leg_coeffs, mu0, phi0, I0
):  # The argument order approximately follows that of Stamnes' DISORT
    NLeg = len(Leg_coeffs)
    # We require NQuad to be >2, even and <= NLeg
    assert NQuad > 2
    assert NQuad % 2 == 0
    assert NQuad <= NLeg
    # We assume that NQuad is even
    N = NQuad // 2

    # For positive mu values (the weights are identical for both domains)
    mu_arr_pos, weights = legendre.leggauss(N)
    mu_arr_pos = mu_arr_pos * (1 / 2) + (1 / 2)
    weights = weights / 2
    # For negative mu values
    mu_arr_neg = -mu_arr_pos

    if not only_flux:
        GC_collect = np.empty((NQuad, NQuad, NLeg))
        eigenvals_collect = np.empty((NQuad, NLeg))
        B_collect = np.empty((NQuad, NLeg))

    not_complex = True
    for m in range(NLeg):
        D_pos, D_neg = generate_Ds(m)
        # We use broadcasting instead of diagonal matrices for computational efficiency
        M_inv = 1 / mu_arr_pos
        W = weights[None, :]
        alpha = M_inv[:, None] * (D_pos * W - np.eye(N))
        beta = M_inv[:, None] * D_neg * W
        A = np.vstack((np.hstack((-alpha, -beta)), np.hstack((beta, alpha))))

        eigenvals_squared, eigenvecs_GpG = np.linalg.eig(
            (alpha - beta) @ (alpha + beta)
        )
        eigenvals = np.concatenate(
            (
                np.sqrt(eigenvals_squared.astype(complex)),
                -np.sqrt(eigenvals_squared.astype(complex)),
            )
        )
        # The eigenvalues are often, but not always real. It is more computationally efficient to work with real values.
        # Since the matrix is real, we can obtain a real eigenvector if the corresponding eigenvalue is real, and
        # the eigenvector will be complex if the corresponding eigenvalue is complex.
        if np.allclose(np.imag(eigenvals), 0):
            eigenvals = np.real(eigenvals)
        elif not_complex and not only_flux:
            eigenvals_collect = eigenvals_collect.astype(complex)
            GC_collect = GC_collect.astype(complex)
            not_complex = False

        eigenvecs_GpG = np.hstack((eigenvecs_GpG, eigenvecs_GpG))
        eigenvecs_GmG = (alpha + beta) @ eigenvecs_GpG / -eigenvals

        G_pos = (eigenvecs_GpG + eigenvecs_GmG) / 2
        G_neg = (eigenvecs_GpG - eigenvecs_GmG) / 2
        G = np.vstack((G_pos, G_neg))

        X_pos, X_neg = generate_Xs(m)
        X_tilde = np.concatenate((-M_inv * X_pos, M_inv * X_neg))

        B = np.linalg.solve(-(np.eye(NQuad) / mu0 + A), X_tilde)
        B_pos, B_neg = B[:N], B[N:]
        Dk_tau_prime = np.exp(eigenvals * tau_prime)

        LHS = np.vstack((G_pos * Dk_tau_prime[None, :], G_neg))
        RHS = np.concatenate((b_pos - B_pos * np.exp(-tau_prime / mu0), b_neg - B_neg))
        C = np.linalg.solve(LHS, RHS)

        if only_flux:

            def flux_up(tau):
                tau = np.atleast_1d(tau)
                um = (G_pos * C[None, :]) @ np.exp(np.outer(eigenvals, tau)) + np.outer(
                    B_pos, np.exp(-tau / mu0)
                )
                return np.squeeze(2 * pi * (mu_arr_pos * weights) @ um)

            def flux_down(tau):
                tau = np.atleast_1d(tau)
                um = (G_neg * C[None, :]) @ np.exp(np.outer(eigenvals, tau)) + np.outer(
                    B_neg, np.exp(-tau / mu0)
                )
                return np.squeeze(2 * pi * (mu_arr_pos * weights) @ um)

            return np.concatenate((mu_arr_pos, mu_arr_neg)), flux_up, flux_down

        GC_collect[:, :, m] = G * C[None, :]
        eigenvals_collect[:, m] = eigenvals
        B_collect[:, m] = B

    def flux_up(tau):
        tau = np.atleast_1d(tau)
        um = GC_collect[:N, :, 0] @ np.exp(
            np.outer(eigenvals_collect[:, 0], tau)
        ) + np.outer(B_collect[:N, 0], np.exp(-tau / mu0))
        return np.squeeze(2 * pi * (mu_arr_pos * weights) @ um)

    def flux_down(tau):
        tau = np.atleast_1d(tau)
        um = GC_collect[N:, :, 0] @ np.exp(
            np.outer(eigenvals_collect[:, 0], tau)
        ) + np.outer(B_collect[N:, 0], np.exp(-tau / mu0))
        return np.squeeze(2 * pi * (mu_arr_pos * weights) @ um)

    def u(tau, phi):
        tau = np.atleast_1d(tau)
        # um must be real; the second term is already real
        um = np.real(
            np.einsum(
                "ijm, jmt -> imt",
                GC_collect,
                np.exp(np.tensordot(eigenvals_collect, tau, axes=0)),
                optimize=True,
            )
        ) + np.tensordot(B_collect, np.exp(-tau / mu0), axes=0)

        return np.squeeze(
            np.tensordot(um, np.cos(np.outer(np.arange(NLeg), phi0 - phi)), axes=(1, 0))
        )

    return np.concatenate((mu_arr_pos, mu_arr_neg)), u, flux_up, flux_down

===================================================== End of algorithm =====================================================

## 2.3 Breakdown and verification of PyDISORT

Generation of Gauss-Legendre quadrature weights and points to numerically integrate over $\mu$ from $-1$ to $1$

In [17]:
# We assume that NQuad is even
N = NQuad // 2

# For positive mu values (the weights are identical for both domains)
mu_arr_pos, weights = legendre.leggauss(N)
mu_arr_pos = mu_arr_pos * (1 / 2) + (1 / 2)
weights = weights / 2

# For negative mu values
mu_arr_neg = -mu_arr_pos

Algorithm to generate Clenshaw-Curtis quadrature weights and points for numerical integration over $\phi$. Required for tests.

In [18]:
def Clenshaw_Curtis_quad(Nphi, a=0, b=2 * pi):
    # Ensure that the number of nodes is odd and greater than 2
    assert Nphi > 2
    assert Nphi % 2 == 1

    Nphi -= 1  # The extra index corresponds to the point 0 which we will add later
    Nphi_pos = Nphi // 2
    phi_arr_pos = np.cos(pi * np.arange(Nphi_pos) / Nphi)
    phi_arr = np.hstack((phi_arr_pos, 0, -phi_arr_pos))
    d = np.hstack((2, 2 / (1 - 4 * np.arange(1, Nphi_pos + 1) ** 2)))
    pos_weights_phi = sc.fft.idct(d, type=1)
    pos_weights_phi[0] /= 2
    full_weights_phi = np.hstack((pos_weights_phi, pos_weights_phi[:-1]))
    return phi_arr * ((b - a) / 2) + ((b + a) / 2), full_weights_phi * ((b - a) / 2)

**Verification of quadrature weights and points on test integral**

$$\int_{a}^{b} e^x \ \mathrm{d}x = e^b - e^a$$

In [19]:
# Gauss-Legendre quadrature; integrate from -1 to 1
mu_arr = np.concatenate((mu_arr_pos, mu_arr_neg))
full_weights_mu = np.concatenate((weights, weights))

true_sol = np.exp(1) - np.exp(-1)
print(
    "Gauss-Legendre quadrature % error =",
    np.abs((true_sol - np.sum(np.exp(mu_arr) * full_weights_mu)) / true_sol),
)

Gauss-Legendre quadrature % error = 0.0


In [20]:
# Clenshaw-Curtis quadrature; integrate from 0 to 2pi
phi_arr, full_weights_phi = Clenshaw_Curtis_quad(33)

true_sol = np.exp(2*pi) - 1
print(
    "Clenshaw-Curtis quadrature % error =",
    np.abs((true_sol - np.sum(np.exp(phi_arr) * full_weights_phi)) / true_sol),
)

Clenshaw-Curtis quadrature % error = 2.1270086547936496e-16


**Normalization verification of `p_HG`**

Similar to equation (2) of [[6]](#cite-MW1980), we expect

$$
\frac{1}{4 \pi} \int_{-1}^1 \int_0^{2 \pi} p\left(\mu, \phi ; \mu^{\prime}, \phi^{\prime}\right) d \phi d \mu = 1
$$

In [21]:
phi_arr, full_weights_phi = Clenshaw_Curtis_quad(33)
mu_arr = np.concatenate((mu_arr_pos, mu_arr_neg))
full_weights_mu = np.concatenate((weights, weights))

normalize_pHG = np.tensordot(
    np.tensordot(
        p_HG_muphi(mu_arr, phi_arr, mu_arr, phi_arr), full_weights_mu, axes=(0, 0)
    ),
    full_weights_phi,
    axes=(0, 0),
) / (4 * pi)
print("L_inf % error =", np.linalg.norm(normalize_pHG - 1, ord=np.infty))

L_inf % error = 0.062211405074487525


### 2.3.1 Re-derivation of equations (6a) to (6d) in [[1]](#cite-STWJ1988)

We have the definitions and expansions

$$
\begin{align*}
u\left(\tau, \mu, \phi\right) &\approx \sum_{n=0} u^n\left(\tau, \mu\right)\cos\left(n\left(\phi_0 - \phi\right)\right) \quad \mbox{(Fourier cosine expansion)}\\
p\left(\cos\gamma\right) &\approx \sum_{\ell=0} g_\ell P_\ell\left(\cos\gamma\right)
\end{align*}
$$
$$
\begin{aligned}
g_\ell &= \frac{2\ell + 1}{2}\int_{-1}^{1} p\left(\cos\gamma\right) P_\ell\left(\cos\gamma\right) \mathrm{d}\cos\gamma, \quad &&g_\ell^m = \frac{\left(\ell-m\right)!}{\left(\ell+m\right)!} g_\ell \\
\mu &= \cos\left(\theta\right), \quad &&\,\mu' = \cos\left(\theta'\right)
\end{aligned}
$$

As before, we have

$$
\begin{align*}
\cos\gamma &= \cos\theta'\cos\theta + \sin\theta'\sin\theta\cos\left(\phi'-\phi\right) \\
P_\ell\left(\cos\gamma\right) &= P_\ell\left(\mu'\right)P_\ell\left(\mu\right) + 2\sum_{m=1}^\ell \frac{\left(\ell-m\right)!}{\left(\ell+m\right)!}P_\ell^m(\mu')P_\ell^m\left(\mu\right)\cos\left(m\left(\phi'-\phi\right)\right)
\end{align*}
$$

Consequently, we can expand the `p_HG_muphi` form of the HG phase function

$$
p\left(\mu, \phi; \mu', \phi'\right) \approx \sum_{\ell=0} \left[ g_\ell P_\ell\left(\mu'\right)P_\ell\left(\mu\right) + 2\sum_{m=1}^\ell g_\ell^m P_\ell^m(\mu')P_\ell^m\left(\mu\right)\cos\left(m\left(\phi'-\phi\right)\right) \right]
$$

We will first focus on the double integral term of the radiative transfer equation. We substitute the expansion of $p(\mu, \phi; \mu', \phi')$ and $u(\tau, \mu, \phi)$ to get

$$
\begin{align*}
&\frac{\omega_0}{4 \pi} \int_{-1}^{1} \int_{0}^{2 \pi} p\left(\mu, \phi ; \mu', \phi'\right) u\left(\tau, \mu', \phi'\right) \mathrm{d} \phi' \mathrm{d} \mu' \\
&\approx \frac{\omega_0}{4 \pi} \int_{-1}^{1} \left[ \int_{0}^{2 \pi} \sum_{n=0} \sum_{\ell=0} u^n g_\ell P_\ell\left(\mu'\right)P_\ell(\mu) \cos\left(n\left(\phi_0 - \phi'\right)\right) + 2\sum_{n=0} \sum_{\ell=0} \sum_{m=1}^\ell u^n g_\ell^m P_\ell^m(\mu')P_\ell^m(\mu)\cos\left(m\left(\phi'-\phi\right)\right) \cos\left(n\left(\phi_0 - \phi'\right)\right) \mathrm{d} \phi' \right] \mathrm{d} \mu' \\
&= \frac{\omega_0}{4 \pi} \int_{-1}^{1} \left[ \int_{0}^{2 \pi} \sum_{\ell=0} u^0 g_\ell P_\ell\left(\mu'\right)P_\ell(\mu) + 2\sum_{n=1} \sum_{\ell=n} \sum_{m=1}^\ell u^n g_\ell^m P_\ell^m(\mu')P_\ell^m(\mu)\cos\left(m\left(\phi'-\phi\right)\right) \cos\left(n\left(\phi_0 - \phi'\right)\right) \mathrm{d} \phi' \right] \mathrm{d} \mu' \\
&= \frac{\omega_0}{4 \pi} \int_{-1}^{1} \left[ 2\pi \sum_{\ell=0} u^0 g_\ell P_\ell\left(\mu'\right)P_\ell(\mu) + 2\pi\sum_{n=1} \sum_{\ell=n} u^n g_\ell^n P_\ell^n(\mu')P_\ell^n(\mu)\cos\left(n\left(\phi_0 - \phi\right)\right) \right] \mathrm{d} \mu' \\
&= \int_{-1}^{1} \sum_{m=0} \left\{ \frac{\omega_0}{2} \sum_{\ell=m} u^m g_\ell^m P_\ell^m(\mu')P_\ell^m(\mu) \right\} \cos\left(m\left(\phi_0 - \phi\right)\right) \mathrm{d} \mu'
\end{align*}
$$

The term in the curly brackets of the last line is the contribution of the double-integral term to the $m$th Fourier moment of the radiative transfer equation.

Next, we will focus on the source term. Once again, we substitute the expansion of $p(\mu, \phi; \mu', \phi')$ to get

$$
\frac{\omega_0 I_0}{4 \pi} p\left(\mu, \phi ;-\mu_{0}, \phi_{0}\right) \exp\left(-\mu_{0}^{-1} \tau\right) \approx \frac{\omega_0 I_0}{4 \pi} \exp\left(-\mu_{0}^{-1} \tau\right) \left[ \sum_{\ell=0} g_\ell P_\ell\left(-\mu_0\right)P_\ell(\mu) + 2\sum_{\ell=0}\sum_{m=1}^\ell g_\ell^m P_\ell^m\left(-\mu_0\right)P_\ell^m(\mu)\cos\left(m\left(\phi_0-\phi\right)\right) \right]
$$

It is immediately apparent that the contribution to the $0$th moment is

$$
\frac{\omega_0 I_0}{4 \pi} \exp\left(-\mu_{0}^{-1} \tau\right)\sum_{\ell=0} g_\ell P_\ell\left(-\mu_0\right)P_\ell(\mu)
$$

For $n \geq 1$, to determine the contribution to the $n$th moment, we multiply by $\pi^{-1}\cos\left(n\left(\phi_0-\phi\right)\right)$ and integrate over $\phi$ from $0$ to $2\pi$ to get

$$
\begin{align*}
&\frac{\omega_0 I_0}{4 \pi} \exp\left(-\mu_{0}^{-1} \tau\right) \int_{0}^{2\pi} \frac{2}{\pi}\sum_{\ell=0}\sum_{m=1}^\ell g_\ell^m P_\ell^m\left(-\mu_0\right)P_\ell^m(\mu)\cos\left(m\left(\phi_0-\phi\right)\right)\cos\left(n\left(\phi_0-\phi\right)\right) \mathrm{d}\phi \\
&= \frac{\omega_0 I_0}{4 \pi} \exp\left(-\mu_{0}^{-1} \tau\right) \int_{0}^{2\pi} \frac{2}{\pi}\sum_{\ell=n}\sum_{m=1}^\ell g_\ell^m P_\ell^m\left(-\mu_0\right)P_\ell^m(\mu)\cos\left(m\left(\phi_0-\phi\right)\right)\cos\left(n\left(\phi_0-\phi\right)\right) \mathrm{d}\phi \\
&= \frac{\omega_0 I_0}{2 \pi} \exp\left(-\mu_{0}^{-1} \tau\right) \sum_{\ell=n} g_\ell^n P_\ell^n\left(-\mu_0\right)P_\ell^n(\mu)
\end{align*}
$$

Therefore, the contribution of the source term to the $m$th Fourier moment of the radiative transfer equation (we perform the change of variables $m = n$) is

$$
\kappa_m\exp\left(-\mu_{0}^{-1} \tau\right)\sum_{\ell=0} g^m_\ell P_\ell^m\left(-\mu_0\right)P_\ell^m(\mu), \quad \kappa_m = \begin{cases} \frac{\omega_0 I_0}{4 \pi}, &m = 0 \\ \frac{\omega_0 I_0}{2 \pi}, &m \geq 1 \end{cases}
$$

Consequently, for each Fourier mode, $m \geq 0$, we have the integro-differential equation

$$
\mu \frac{d u^m(\tau, \mu)}{d \tau}=u^m(\tau, \mu)-\int_{-1}^1 D^m\left(\tau, \mu, \mu^{\prime}\right) u^m\left(\tau, \mu^{\prime}\right) d \mu^{\prime} - Q^m(\mu)
$$

where

$$
\begin{align*}
D^m\left(\mu, \mu' \right) &= \frac{\omega_0}{2} \sum_{\ell=m} u^m g_\ell^m P_\ell^m(\mu')P_\ell^m(\mu) \\
Q^m(\mu) &= X^m(\mu) \exp\left(-\mu_{0}^{-1} \tau\right) \\
X^m(\mu) &= \kappa_m\sum_{\ell=0} g^m_\ell P_\ell^m\left(-\mu_0\right)P_\ell^m(\mu), \quad \kappa_m = \begin{cases} \frac{\omega_0 I_0}{4 \pi}, &m = 0 \\ \frac{\omega_0 I_0}{2 \pi}, &m \geq 1 \end{cases}
\end{align*}
$$

Unlike in [[1]](#cite-STWJ1988), our source term, $Q$, only contains the "beam" term and not a "thermal" term. It is also only dependent on $\mu$.

**This is the non-vectorized version of the `generate_Ds` subroutine for verification**

In [22]:
for m in range(NLeg):
    D_pos_test, D_neg_test = np.zeros((N, N)), np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            for ell in range(m, NLeg):
                D_pos_test[i, j] += (
                    (w0 / 2)
                    # * (2 * ell + 1) # Moved into the main algorithm
                    * Leg_coeffs[ell]
                    * (sc.special.factorial(ell - m) / sc.special.factorial(ell + m))
                    * sc.special.lpmv(m, ell, mu_arr_pos[i])
                    * sc.special.lpmv(m, ell, mu_arr_pos[j])
                )
                D_neg_test[i, j] += (
                    (w0 / 2)
                    # * (2 * ell + 1) # Moved into the main algorithm
                    * Leg_coeffs[ell]
                    * (sc.special.factorial(ell - m) / sc.special.factorial(ell + m))
                    * sc.special.lpmv(m, ell, mu_arr_neg[i])
                    * sc.special.lpmv(m, ell, mu_arr_pos[j])
                )

    assert np.allclose(D_pos_test, generate_Ds(m)[0])
    assert np.allclose(D_neg_test, generate_Ds(m)[1])

print("Passed all tests")

Passed all tests


**This is the non-vectorized version of the `generate_Xs` subroutine (for the non-trivial case $m = 0$) for verification**

In [23]:
for m in range(NLeg):
    if m == 0:
        prefactor = w0 * I0 / (4 * pi)
    else:
        prefactor = w0 * I0 / (2 * pi)
    X_pos_test, X_neg_test = np.zeros(N), np.zeros(N)
    for i in range(N):
        for ell in range(m, NLeg):
            X_pos_test[i] += (
                prefactor
                * Leg_coeffs[ell]
                * (sc.special.factorial(ell - m) / sc.special.factorial(ell + m))
                * sc.special.lpmv(m, ell, -mu0)
                * sc.special.lpmv(m, ell, mu_arr_pos[i])
            )
            X_neg_test[i] += (
                prefactor
                * Leg_coeffs[ell]
                * (sc.special.factorial(ell - m) / sc.special.factorial(ell + m))
                * sc.special.lpmv(m, ell, -mu0)
                * sc.special.lpmv(m, ell, mu_arr_neg[i])
            )

    assert np.allclose(X_pos_test, generate_Xs(m)[0])
    assert np.allclose(X_neg_test, generate_Xs(m)[1])

print("Passed all tests")

Passed all tests


### 2.3.2 Re-derivation of equations (7a) and (7b) in [[1]](#cite-STWJ1988)

We split the $\mu$ integral into two integrals: from $-1$ to $0$ and from $0$ to $1$. We approximate each integral by Gauss-Legendre quadrature. This is the *double-Gauss method*; see [[7]](#cite-Syk1951) for more details. By double-Gauss, or any other double quadrature method (more general quadrature methods can used, but we will lose important symmetries), we can approximate each Fourier mode integro-differential equation as


$$
\mu_i \frac{d u^m(\tau, \mu_i)}{d \tau}=u^m(\tau, \mu_i)-\sum_{j \neq 0} w_j D^m\left(\tau, \mu_i, \mu_i^{\prime}\right) u^m\left(\tau, \mu_i^{\prime}\right) d \mu_i^{\prime} - Q^m(\mu_i)
$$

For $i,j = 1, \dots, N$, where $2N$ is the number of quadrature points, we define

$$
\begin{aligned}
&\alpha = M^{-1}\left(D^{+} W - I\right) &&\beta = M^{-1} D^{-} W \\
&D^{+}[i,j] = D^m\left(\mu_i, \mu_j\right) = D^m\left(-\mu_i,-\mu_j\right) &&D^{-}[i,j] = D^m\left(-\mu_i, \mu_j\right) = D^m\left(\mu_i,-\mu_j\right) \\
&W[i,j] = w_i\delta_{ij} &&M[i,j] = \mu_i\delta_{ij} \\ 
&u^\pm[i] = u^m(\pm \mu_i) &&Q^{\pm}[i] = Q^m\left(\pm \mu_i\right) 
\end{aligned}
$$

We also define $\tilde{Q}^\pm = M^{-1} Q^\pm$. We claim that the Fourier mode approximations can be re-expressed as the system

$$
\begin{bmatrix} \frac{\mathrm{d}u^+}{\mathrm{d}\tau} \\ \frac{\mathrm{d}u^-}{\mathrm{d}\tau} \end{bmatrix} = \begin{bmatrix} -\alpha & -\beta \\ \beta & \alpha \end{bmatrix} \begin{bmatrix} u^+ \\ u^- \end{bmatrix} + \begin{bmatrix} -\tilde{Q}^+ \\ \tilde{Q}^- \end{bmatrix}
$$

Substitute $\alpha, \beta, \tilde{Q}$ on the RHS:

$$
\begin{align*}
\begin{bmatrix} -\alpha & -\beta \\ \beta & \alpha \end{bmatrix} \begin{bmatrix} u^+ \\ u^- \end{bmatrix} + \begin{bmatrix} -\tilde{Q}^+ \\ \tilde{Q}^- \end{bmatrix} &= \begin{bmatrix} -M^{-1}\left(D^{+} W - I\right) & -M^{-1} D^{-} W \\ M^{-1} D^{-} W & M^{-1}\left(D^{+} W - I\right) \end{bmatrix} \begin{bmatrix} u^+ \\ u^- \end{bmatrix} + \begin{bmatrix} -M^{-1} Q^+ \\ M^{-1} Q^- \end{bmatrix} \\
&= \begin{bmatrix} -M^{-1} & \\ & M^{-1} \end{bmatrix} \left( \begin{bmatrix} D^{+} W - I & D^{-} W \\ D^{-} W & D^{+} W - I \end{bmatrix} \begin{bmatrix} u^+ \\ u^- \end{bmatrix} + \begin{bmatrix} Q^+ \\ Q^- \end{bmatrix} \right) \\
&= \begin{bmatrix} -\mu_0^{-1} & & & & & \\ & \ddots & & & & \\ & & -\mu_N^{-1} & & & \\ & & & \mu_0^{-1} & & \\ & & & & \ddots & \\ & & & & & \mu_N^{-1} \end{bmatrix} \left( \left( \begin{bmatrix} D^{+} W & D^{-} W \\ D^{-} W & D^{+} W \end{bmatrix} - I \right) \begin{bmatrix} u^+ \\ u^- \end{bmatrix} + \begin{bmatrix} Q^+ \\ Q^- \end{bmatrix} \right) \\
&= \begin{bmatrix} \mu_0^{-1} & & & & & \\ & \ddots & & & & \\ & & \mu_N^{-1} & & & \\ & & & -\mu_0^{-1} & & \\ & & & & \ddots & \\ & & & & & -\mu_N^{-1} \end{bmatrix} \left(\begin{bmatrix} u^+ \\ u^- \end{bmatrix} - W \begin{bmatrix} D^{+} & D^{-} \\ D^{-} & D^{+} \end{bmatrix} - \begin{bmatrix} Q^+ \\ Q^- \end{bmatrix} \right)
\end{align*}
$$

Finally, we multiply across by the $\mu_i$ values to see that the system is consistent with the Fourier mode approximations.

### 2.3.3 Solving the system for each Fourier mode

As previously derived, the system is

$$
\begin{bmatrix} \frac{\mathrm{d}u^+}{\mathrm{d}\tau} \\ \frac{\mathrm{d}u^-}{\mathrm{d}\tau} \end{bmatrix} = \begin{bmatrix} -\alpha & -\beta \\ \beta & \alpha \end{bmatrix} \begin{bmatrix} u^+ \\ u^- \end{bmatrix} + \begin{bmatrix} -\tilde{Q}^+ \\ \tilde{Q}^- \end{bmatrix}
$$

Define $\tilde{Q} = \begin{bmatrix} -\tilde{Q}^+ \tilde{Q}^- \end{bmatrix}^T$ and $\tilde{X} = \exp\left(\mu_0^{-1} \tau\right)\tilde{Q}$. We first address the homogeneous problem, when $\tilde{Q} = 0$, and solve for the eigenpairs of the coefficient matrix from the eigenequation

$$
\begin{bmatrix} -\alpha & -\beta \\ \beta & \alpha \end{bmatrix} \begin{bmatrix} G^+ \\ G^- \end{bmatrix} = k \begin{bmatrix} G^+ \\ G^- \end{bmatrix}
$$

Following the reduction order method in [[1]](#cite-STWJ1988), but with minor sign differences, we can solve

$$
(\alpha - \beta) (\alpha + \beta) \left(G^+ + G^-\right) = k^2 \left(G^+ + G^-\right)
$$

for eigenvalues $k$. Given also that

$$
(\alpha + \beta) \left(G^+ + G^-\right) = -k \left(G^+ - G^-\right)
$$

we can solve for $G^+$ and $G^-$, and consequently construct the eigenvector matrix $G = \begin{bmatrix} G^+ & G^- \end{bmatrix}^T$.

In [24]:
m = 3  # We will need to repeat the following blocks of code for each Fourier mode, m

In [25]:
D_pos, D_neg = generate_Ds(m)
# We use broadcasting instead of diagonal matrices for computational efficiency
M_inv = 1 / mu_arr_pos
W = weights[None, :]
alpha = M_inv[:, None] * (D_pos * W - np.eye(N))
beta = M_inv[:, None] * D_neg * W
A = np.vstack((np.hstack((-alpha, -beta)), np.hstack((beta, alpha))))

eigenvals_squared, eigenvecs_GpG = np.linalg.eig((alpha - beta) @ (alpha + beta))
eigenvals = np.concatenate(
    (
        np.sqrt(eigenvals_squared.astype(complex)),
        -np.sqrt(eigenvals_squared.astype(complex)),
    )
)
# The eigenvalues are often, but not always real. It is more computationally efficient to work with real values.
# Since the matrix is real, we can obtain a real eigenvector if the corresponding eigenvalue is real, and
# the eigenvector will be complex if the corresponding eigenvalue is complex.
if np.allclose(np.imag(eigenvals), 0):
    eigenvals = np.real(eigenvals)

eigenvecs_GpG = np.hstack((eigenvecs_GpG, eigenvecs_GpG))
eigenvecs_GmG = (alpha + beta) @ eigenvecs_GpG / -eigenvals

G_pos = (eigenvecs_GpG + eigenvecs_GmG) / 2
G_neg = (eigenvecs_GpG - eigenvecs_GmG) / 2
G = np.vstack((G_pos, G_neg))

**Verification of eigenpairs**

In [26]:
assert np.allclose((A @ G) / eigenvals, G)

print("Passed all tests")

Passed all tests


### 2.3.4 Constructing the general solution for each Fourier mode

The general solution is

$$
u = u_h + u_p
$$

The particular solution, $u_p$, satisfies

$$
\frac{\mathrm{d}u_p}{\mathrm{d}\tau} = A u_p + \tilde{Q}
$$

where

$$
A = \begin{bmatrix} -\alpha & -\beta \\ \beta & \alpha \end{bmatrix}, \quad \tilde{Q} = \tilde{X} \exp\left(-\mu_0^{-1} \tau\right)
$$

Assume the ansatz

$$
u_p = B\exp\left(-\mu_0^{-1} \tau\right)
$$

for $B$ to be determined. Substitution into the full equation gives

\begin{align*}
&-\mu_0^{-1} B\exp\left(-\mu_0^{-1} \tau\right) = AB\exp\left(-\mu_0^{-1} \tau\right) + \tilde{X}\exp\left(-\mu_0^{-1} \tau\right) \\
&\implies -\mu_0^{-1} B = AB + \tilde{X} \\
&\implies -\left(\mu_0^{-1} I + A\right)B = \tilde{X}
\end{align*}

which we can solve for $B$.

In [27]:
X_pos, X_neg = generate_Xs(m)
X_tilde = np.concatenate((-M_inv * X_pos, M_inv * X_neg))

B = np.linalg.solve(-(np.eye(NQuad) / mu0 + A), X_tilde)

**Verification of particular solution**

In [28]:
Ntau = 2**9  # Number of tau grid points
tau_arr = np.linspace(0, tau_prime, Ntau)
h = tau_arr[1] - tau_arr[0]  # grid spacing

# Construct 1st derivative matrix with 2nd order accuracy
first_deriv = np.zeros((Ntau, Ntau))
diagonal = np.ones(Ntau) / (2 * h)
first_deriv += np.diag(diagonal[:-1], 1)
first_deriv += np.diag(-diagonal[:-1], -1)
first_deriv[0, :3] = np.array([-3 / 2, 2, -1 / 2]) / h
first_deriv[-1, -3:] = np.array([1 / 2, -2, 3 / 2]) / h
first_deriv = first_deriv.T # This is due to tau being indexed by columns instead of rows

In [29]:
up = np.outer(B, np.exp(-tau_arr / mu0))
RHS = up @ first_deriv
LHS = A @ up + np.outer(X_tilde, np.exp(-tau_arr / mu0))

print("Pointwise L_inf % error:", np.linalg.norm((RHS - LHS) / RHS))

Pointwise L_inf % error: 1.3009691165637128e-07


The homogenous solution, $u_h$, is

$$
u_h = \begin{bmatrix} u_h^+ \\ u_h^- \end{bmatrix}, \quad u_h^\pm(\tau) = G^\pm \mbox{Diag}(C) K
$$

where $K[j] = \exp(k_j\tau)$ and the coefficient vector $C$ is to be determined from the boundary conditions. We define $\tau \in [0, \tau']$ and assume Dirichlet BCs (more general BCs are possible but not implemented)

$$
u^-(0) = b^-, \quad u^+_i\left( \tau' \right) = b^+
$$

By superposition

$$
u_h^-(0) = b^- - B^-, \quad u_h^+\left( \tau' \right) = b^+ - B^+\exp\left(-\mu_0^{-1} \tau'\right)
$$

and this produces the system

$$
\begin{bmatrix} G^+ D_k\left( \tau' \right) \\ G^- \end{bmatrix} C = \begin{bmatrix} b^+ - B^+\exp\left(-\mu_0^{-1} \tau'\right) \\ b^- - B^- \end{bmatrix}
$$

which we can solve to determine $C$.

In [30]:
B_pos, B_neg = B[:N], B[N:]
Dk_tau_prime = np.exp(eigenvals * tau_prime)

LHS = np.vstack((G_pos * Dk_tau_prime[None, :], G_neg))
RHS = np.concatenate((b_pos - B_pos * np.exp(-tau_prime / mu0), b_neg - B_neg))
C = np.linalg.solve(LHS, RHS)

#### 2.3.4.1 Verification of the general solution for one Fourier mode

In [31]:
# The general solution for one Fourier mode
def um(tau):
    result = (G * C[None, :]) @ np.exp(np.outer(eigenvals, tau)) + np.outer(
        B, np.exp(-tau / mu0)
    )

    # The general solution must be real
    assert np.allclose(np.imag(result), 0)
    return np.real(result)

**Does the general solution satisfy the BCs?**

In [32]:
# At top of atmosphere
assert np.allclose(um(0)[N:], b_neg)

# At bottom of atmosphere
assert np.allclose(um(tau_prime)[:N], b_pos)

print("Passed all tests")

Passed all tests


**Does the general solution satisfy the system of ODEs?**

In [33]:
um_cache = um(tau_arr)
RHS = um_cache @ first_deriv
LHS = A @ um_cache + np.outer(X_tilde, np.exp(-tau_arr / mu0))

print("Pointwise L_inf % error:", np.linalg.norm((RHS - LHS) / RHS, ord=np.infty))

Pointwise L_inf % error: 0.026988727349968232


**Does the general solution satisfy the Fourier mode integro-differential equation?**

In [34]:
D = np.hstack((np.vstack((D_pos, D_neg)), np.vstack((D_neg, D_pos))))
RHS = mu_arr[:, None] * um(tau_arr) @ first_deriv
LHS = (
    um_cache
    - np.tensordot(
        np.einsum("ij, jt -> ijt", D, um_cache, optimize=True),
        full_weights_mu,
        axes=(1, 0),
    )
    - np.outer(np.concatenate((X_pos, X_neg)), np.exp(-tau_arr / mu0))
)
print("L_inf % error:", np.linalg.norm((RHS - LHS) / RHS, ord=np.infty))

L_inf % error: 0.026988727349061378


### 2.3.5 The full solution and computation of flux, reflectivity, transmittivity

The above must be repeated for each Fourier mode. The full solution given by `PyDISORT` is

$$
u(\tau, \mu, \phi) = \sum_{m=0} u^m(\mu,\tau)\cos\left(m\left(\phi_0 - \phi\right)\right)
$$

This solution is continuous and variable with respect to $\tau$ and $\phi$ but discrete and fixed with respect to $\mu$. The function output is 3-dimensional and axes $0, 1, 2$ capture $\mu, \tau, \phi$ variation respectively. The solution $u$ is easily split into $u^+$ and $u^-$ by halving the $\mu$ index.

`PyDISORT` also returns the positive and negative (hemispheric) flux functions

$$
\begin{align*}
\mbox{Flux}^\pm(\tau) &= \int_{0}^{1} \int_{0}^{2 \pi} \mu u\left(\tau, \pm\mu, \phi\right) \mathrm{d} \phi \mathrm{d} \mu \\
& \approx \sum_{m=0} \left(\int_{0}^{1} \mu u^m(\tau, \pm\mu) \mathrm{d} \mu \int_{0}^{2 \pi} \cos\left(m\left(\phi_0 - \phi\right)\right) \mathrm{d} \phi \right) \\
&= 2\pi \int_{0}^{1} \mu u^0\left(\tau, \pm\mu\right) \mathrm{d} \mu \\
&\approx 2\pi \sum_{i = 1} w_i\mu_i u^0\left(\tau, \pm\mu_i\right)
\end{align*}
$$

In the last line we use Gauss-Legendre quadrature to approximate the $\mu$ integral.

In [35]:
Ntau = 2**9  # Number of tau grid points
Nphi = 2**6 + 1  # Number of phi grid points

tau_arr = np.linspace(0, tau_prime, Ntau)
h = tau_arr[1] - tau_arr[0]  # grid spacing
phi_arr, full_weights_phi = Clenshaw_Curtis_quad(Nphi)

# Construct 1st derivative matrix with 2nd order accuracy
first_deriv = np.zeros((Ntau, Ntau))
diagonal = np.ones(Ntau) / (2 * h)
first_deriv += np.diag(diagonal[:-1], 1)
first_deriv += np.diag(-diagonal[:-1], -1)
first_deriv[0, :3] = np.array([-3 / 2, 2, -1 / 2]) / h
first_deriv[-1, -3:] = np.array([1 / 2, -2, 3 / 2]) / h
first_deriv = first_deriv.T # This is due to tau being indexed by columns instead of rows

mu_arr, u, flux_up, flux_down = PyDISORT(
    b_pos, b_neg, False, NQuad, tau_prime, w0, Leg_coeffs, mu0, phi0, I0
)
full_weights_mu = np.concatenate((weights, weights))

#### 2.3.5.1 Verification of full solution

**Does the full solution satisfy the BCs?**

In [36]:
# At top of atmosphere
assert np.allclose(
    u(0, phi_arr)[N:, :],
    np.tensordot(
        np.repeat(np.atleast_1d(b_neg)[:, None], N // len(np.atleast_1d(b_neg))),
        np.sum(np.cos(np.outer(phi0 - phi_arr, np.arange(NLeg))), axis=1),
        axes=0,
    ),
)

# At bottom of atmosphere
assert np.allclose(
    u(tau_prime, phi_arr)[:N, :],
    np.tensordot(
        np.repeat(np.atleast_1d(b_pos)[:, None], N // len(np.atleast_1d(b_pos))),
        np.sum(np.cos(np.outer(phi0 - phi_arr, np.arange(NLeg))), axis=1),
        axes=0,
    ),
)

print("Passed all tests")

Passed all tests


**Does the full solution satisfy the radiative transfer equation?**

In [37]:
u_cache = u(tau_arr, phi_arr)
LHS = mu_arr[:, None, None] * np.moveaxis(
    np.tensordot(u_cache, first_deriv, axes=(1, 0)),
    source=2,
    destination=1,
)

In [38]:
# WARNING: The integrand is a 5-dimensional tensor and so constructing it 
#          can be computationally intensive depending on parameters
integrand = np.einsum(
    "ijkl, ktl -> itjkl",
    p_HG_muphi(mu_arr, phi_arr, mu_arr, phi_arr),
    u_cache,
    optimize=True,
)
RHS = (
    u_cache
    - (w0 / (4 * pi))
    * np.tensordot(
        np.tensordot(integrand, full_weights_mu, axes=(3, 0)),
        full_weights_phi,
        axes=(3, 0),
    )
    - (w0 * I0 / (4 * pi))
    * np.moveaxis(
        np.tensordot(
            p_HG_muphi(mu_arr, phi_arr, -mu0, phi0), np.exp(-tau_arr / mu0), axes=0
        ),
        source=2,
        destination=1,
    )
)

In [39]:
print("L2 error =", np.linalg.norm(LHS - RHS))
print("% error in L2 norm =", np.linalg.norm(LHS - RHS)/np.linalg.norm(RHS))

L2 error = 0.5791894464735415
% error in L2 norm = 0.00046645793984488186


The pointwise error may be large. This discrepancy is partly due the integrals in the radiative transfer equation; information about individual points tends to be lost upon integration. Fortunately, we are more interested in the flux, which is an aggregated quantity, than in intensity values at specific points.

#### 2.3.5.2 Verification of flux, reflectivity, transmittivity (TODO)

**Does integrating the intensity functions produce the flux functions?**

In [40]:
flux_up_test = np.tensordot(
    np.tensordot(
        mu_arr_pos[:, None, None] * u_cache[:N, :], weights, axes=(0, 0)
    ),
    full_weights_phi,
    axes=(1, 0),
)
flux_down_test = np.tensordot(
    np.tensordot(
        mu_arr_pos[:, None, None] * u_cache[N:, :], weights, axes=(0, 0)
    ),
    full_weights_phi,
    axes=(1, 0),
)

assert np.allclose(flux_up_test, flux_up(tau_arr))
assert np.allclose(flux_down_test, flux_down(tau_arr))

In [41]:
print("Upwelling and downwelling =", flux_up(0), flux_down(tau_prime))

Upwelling and downwelling = 0.015779198843884804 0.17074312408273246


*TODO: Calculate Reflectivity and Transmitivity (See [[6]](#cite-MW1980)). Verify that they are equal if $\omega_0 = 1$*

#### 2.3.5.3 Timing PyDISORT

The time taken will of course be parameter-dependent, but this should give a sense of the speed of `PyDISORT`.

**Time taken to solve the radiative transfer equation**

In [42]:
# For intensity
%timeit PyDISORT(b_pos, b_neg, False, NQuad, tau_prime, w0, Leg_coeffs, mu0, phi0, I0)

# For flux
%timeit PyDISORT(b_pos, b_neg, True, NQuad, tau_prime, w0, Leg_coeffs, mu0, phi0, I0)

28.2 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.36 ms ± 41.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**Time taken to evaluate the solution at a point**

In [43]:
# For intensity
%timeit u(tau_arr[Ntau//2], phi_arr[Nphi//2])

# For flux
%timeit flux_up(0)
%timeit flux_down(tau_prime)

94.4 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
14 µs ± 756 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
13.3 µs ± 63.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


<!--bibtex


@book{Cha1960, 
      author = "S.  Chandrasekhar",
      title = "Radiative Transfer",
      year = "1960",
      publisher = "Dover",
}

@article{Wis1977,
      author = "W. J.  Wiscombe",
      title = "The Delta–M Method: Rapid Yet Accurate Radiative Flux Calculations for Strongly Asymmetric Phase Functions",
      journal = "Journal of Atmospheric Sciences",
      year = "1977",
      publisher = "American Meteorological Society",
      address = "Boston MA, USA",
      volume = "34",
      number = "9",
      doi = "10.1175/1520-0469(1977)034<1408:TDMRYA>2.0.CO;2",
      pages=      "1408 - 1422",
      url = "https://journals.ametsoc.org/view/journals/atsc/34/9/1520-0469_1977_034_1408_tdmrya_2_0_co_2.xml"
}

@article{Syk1951,
    author = {Sykes, J. B.},
    title = "{Approximate Integration of the Equation of Transfer}",
    journal = {Monthly Notices of the Royal Astronomical Society},
    volume = {111},
    number = {4},
    pages = {377-386},
    year = {1951},
    month = {08},
    abstract = "{The value of numerical integration in obtaining approximate solutions of an equation of transfer, and the different methods at our disposal, are discussed. It is shown that although the Newton-Cotes method, used by Kourganoff, is better than the Gauss method, used by Chandrasekhar, both are inferior to a new method, the double-Gauss, discovered by the author. The errors in the approximate values of the source-function and the limb-darkening in all three methods are tabulated for various approximations, and illustrated by graphs.}",
    issn = {0035-8711},
    doi = {10.1093/mnras/111.4.377},
    url = {https://doi.org/10.1093/mnras/111.4.377},
    eprint = {https://academic.oup.com/mnras/article-pdf/111/4/377/8077435/mnras111-0377.pdf},
}


@article{STWJ1988,
author = {Knut Stamnes and S-Chee Tsay and Warren Wiscombe and Kolf Jayaweera},
journal = {Appl. Opt.},
keywords = {Electromagnetic radiation; Multiple scattering; Optical depth; Radiative transfer; Reflection; Thermal emission},
number = {12},
pages = {2502--2509},
publisher = {Optica Publishing Group},
title = {Numerically stable algorithm for discrete-ordinate-method radiative transfer in multiple scattering and emitting layered media},
volume = {27},
month = {Jun},
year = {1988},
url = {http://opg.optica.org/ao/abstract.cfm?URI=ao-27-12-2502},
doi = {10.1364/AO.27.002502},
abstract = {We summarize an advanced, thoroughly documented, and quite general purpose discrete ordinate algorithm for time-independent transfer calculations in vertically inhomogeneous, nonisothermal, plane-parallel media. Atmospheric applications ranging from the UV to the radar region of the electromagnetic spectrum are possible. The physical processes included are thermal emission, scattering, absorption, and bidirectional reflection and emissionat the lower boundary. The medium may be forced at the top boundary by parallel or diffuse radiation and by internal and boundary thermal sources as well. We provide a brief account of the theoretical basis as well as a discussion of the numerical implementation of the theory. The recent advances made by ourselves and our collaborators---advances in both formulation and numerical solution---are all incorporated in the algorithm. Prominent among these advances are the complete conquest of two ill-conditioning problems which afflicted all previous discrete ordinate implementations: (1) the computation of eigenvalues and eigenvectors and (2) the inversion of the matrix determining the constants of integration. Copies of the fortran program on microcomputer diskettes are available for interested users.},
}



@article{STL2000,
author = {Stamnes, Knut and Tsay, Si-Chee and Wiscombe, Warren and Laszlo, Istvan and Einaudi, Franco},
year = {2000},
month = {02},
pages = {},
title = {General Purpose Fortran Program for Discrete-Ordinate-Method Radiative Transfer in Scattering and Emitting Layered Media: An Update of DISORT}
}

@article{SS1981,
      author = "Knut  Stamnes and Roy A.  Swanson",
      title = "A New Look at the Discrete Ordinate Method for Radiative Transfer Calculations in Anisotropically Scattering Atmospheres",
      journal = "Journal of Atmospheric Sciences",
      year = "1981",
      publisher = "American Meteorological Society",
      address = "Boston MA, USA",
      volume = "38",
      number = "2",
      doi = "10.1175/1520-0469(1981)038<0387:ANLATD>2.0.CO;2",
      pages=      "387 - 399",
      url = "https://journals.ametsoc.org/view/journals/atsc/38/2/1520-0469_1981_038_0387_anlatd_2_0_co_2.xml"
}

@article{SC1984,
title = {A new multi-layer discrete ordinate approach to radiative transfer in vertically inhomogeneous atmospheres},
journal = {Journal of Quantitative Spectroscopy and Radiative Transfer},
volume = {31},
number = {3},
pages = {273-282},
year = {1984},
issn = {0022-4073},
doi = {https://doi.org/10.1016/0022-4073(84)90031-1},
url = {https://www.sciencedirect.com/science/article/pii/0022407384900311},
author = {Knut Stamnes and Paul Conklin},
abstract = {A recently developed matrix formulation of the discrete ordinate method is extended for application to an inhomogeneous atmosphere. The solution yields fluxes, as well as the complete azimuthal dependence of the intensity at any level in the atmosphere. The numerical aspects of the solution are discussed and numerical verification is provided by comparing computed results with those obtained by other methods. In particular, it is shown that a simple scaling scheme, which removes the positive exponentials in the coefficient matrix when solving for the constants of integration, provides unconditionally stable solutions for arbitrary optical thicknesses. An assessment of the accuracy to be expected is also provided, and it is shown that low-order discrete ordinate approximations yield very accurate flux values.}
}

@article{MH2017,
title = {A demonstration of adjoint methods for multi-dimensional remote sensing of the atmosphere and surface},
journal = {Journal of Quantitative Spectroscopy and Radiative Transfer},
volume = {204},
pages = {215-231},
year = {2018},
issn = {0022-4073},
doi = {https://doi.org/10.1016/j.jqsrt.2017.09.031},
url = {https://www.sciencedirect.com/science/article/pii/S0022407317305198},
author = {William G.K. Martin and Otto P. Hasekamp},
keywords = {Adjoint methods, Three-dimensional vector radiative transfer, Linearization, Remote sensing, Parameter derivatives, Searchlight functions},
abstract = {In previous work, we derived the adjoint method as a computationally efficient path to three-dimensional (3D) retrievals of clouds and aerosols. In this paper we will demonstrate the use of adjoint methods for retrieving two-dimensional (2D) fields of cloud extinction. The demonstration uses a new 2D radiative transfer solver (FSDOM). This radiation code was augmented with adjoint methods to allow efficient derivative calculations needed to retrieve cloud and surface properties from multi-angle reflectance measurements. The code was then used in three synthetic retrieval studies. Our retrieval algorithm adjusts the cloud extinction field and surface albedo to minimize the measurement misfit function with a gradient-based, quasi-Newton approach. At each step we compute the value of the misfit function and its gradient with two calls to the solver FSDOM. First we solve the forward radiative transfer equation to compute the residual misfit with measurements, and second we solve the adjoint radiative transfer equation to compute the gradient of the misfit function with respect to all unknowns. The synthetic retrieval studies verify that adjoint methods are scalable to retrieval problems with many measurements and unknowns. We can retrieve the vertically-integrated optical depth of moderately thick clouds as a function of the horizontal coordinate. It is also possible to retrieve the vertical profile of clouds that are separated by clear regions. The vertical profile retrievals improve for smaller cloud fractions. This leads to the conclusion that cloud edges actually increase the amount of information that is available for retrieving the vertical profile of clouds. However, to exploit this information one must retrieve the horizontally heterogeneous cloud properties with a 2D (or 3D) model. This prototype shows that adjoint methods can efficiently compute the gradient of the misfit function. This work paves the way for the application of similar methods to 3D remote sensing problems.}
}

@article{MCB2014,
title = {Adjoint methods for adjusting three-dimensional atmosphere and surface properties to fit multi-angle/multi-pixel polarimetric measurements},
journal = {Journal of Quantitative Spectroscopy and Radiative Transfer},
volume = {144},
pages = {68-85},
year = {2014},
issn = {0022-4073},
doi = {https://doi.org/10.1016/j.jqsrt.2014.03.030},
url = {https://www.sciencedirect.com/science/article/pii/S002240731400154X},
author = {William Martin and Brian Cairns and Guillaume Bal},
keywords = {Adjoint methods, Three-dimensional vector radiative transfer, Linearization, Remote sensing, Parameter derivatives},
abstract = {This paper derives an efficient procedure for using the three-dimensional (3D) vector radiative transfer equation (VRTE) to adjust atmosphere and surface properties and improve their fit with multi-angle/multi-pixel radiometric and polarimetric measurements of scattered sunlight. The proposed adjoint method uses the 3D VRTE to compute the measurement misfit function and the adjoint 3D VRTE to compute its gradient with respect to all unknown parameters. In the remote sensing problems of interest, the scalar-valued misfit function quantifies agreement with data as a function of atmosphere and surface properties, and its gradient guides the search through this parameter space. Remote sensing of the atmosphere and surface in a three-dimensional region may require thousands of unknown parameters and millions of data points. Many approaches would require calls to the 3D VRTE solver in proportion to the number of unknown parameters or measurements. To avoid this issue of scale, we focus on computing the gradient of the misfit function as an alternative to the Jacobian of the measurement operator. The resulting adjoint method provides a way to adjust 3D atmosphere and surface properties with only two calls to the 3D VRTE solver for each spectral channel, regardless of the number of retrieval parameters, measurement view angles or pixels. This gives a procedure for adjusting atmosphere and surface parameters that will scale to the large problems of 3D remote sensing. For certain types of multi-angle/multi-pixel polarimetric measurements, this encourages the development of a new class of three-dimensional retrieval algorithms with more flexible parametrizations of spatial heterogeneity, less reliance on data screening procedures, and improved coverage in terms of the resolved physical processes in the Earth׳s atmosphere.}
}

@article{LSJLTWS2015,
title = {Improved discrete ordinate solutions in the presence of an anisotropically reflecting lower boundary: Upgrades of the DISORT computational tool},
journal = {Journal of Quantitative Spectroscopy and Radiative Transfer},
volume = {157},
pages = {119-134},
year = {2015},
issn = {0022-4073},
doi = {https://doi.org/10.1016/j.jqsrt.2015.02.014},
url = {https://www.sciencedirect.com/science/article/pii/S0022407315000679},
author = {Z. Lin and S. Stamnes and Z. Jin and I. Laszlo and S.-C. Tsay and W.J. Wiscombe and K. Stamnes},
keywords = {Radiative transfer model, BRDF, Cox–Munk, Ross–Li, RPV, Single scattering correction},
abstract = {A successor version 3 of DISORT (DISORT3) is presented with important upgrades that improve the accuracy, efficiency, and stability of the algorithm. Compared with version 2 (DISORT2 released in 2000) these upgrades include (a) a redesigned BRDF computation that improves both speed and accuracy, (b) a revised treatment of the single scattering correction, and (c) additional efficiency and stability upgrades for beam sources. In DISORT3 the BRDF computation is improved in the following three ways: (i) the Fourier decomposition is prepared “off-line”, thus avoiding the repeated internal computations done in DISORT2; (ii) a large enough number of terms in the Fourier expansion of the BRDF is employed to guarantee accurate values of the expansion coefficients (default is 200 instead of 50 in DISORT2); (iii) in the post-processing step the reflection of the direct attenuated beam from the lower boundary is included resulting in a more accurate single scattering correction. These improvements in the treatment of the BRDF have led to improved accuracy and a several-fold increase in speed. In addition, the stability of beam sources has been improved by removing a singularity occurring when the cosine of the incident beam angle is too close to the reciprocal of any of the eigenvalues. The efficiency for beam sources has been further improved from reducing by a factor of 2 (compared to DISORT2) the dimension of the linear system of equations that must be solved to obtain the particular solutions, and by replacing the LINPAK routines used in DISORT2 by LAPACK 3.5 in DISORT3. These beam source stability and efficiency upgrades bring enhanced stability and an additional 5–7% improvement in speed. Numerical results are provided to demonstrate and quantify the improvements in accuracy and efficiency of DISORT3 compared to DISORT2.}
}

@article {JWW1976,
      author = "J. H.  Joseph and W. J.  Wiscombe and J. A.  Weinman",
      title = "The Delta-Eddington Approximation for Radiative Flux Transfer",
      journal = "Journal of Atmospheric Sciences",
      year = "1976",
      publisher = "American Meteorological Society",
      address = "Boston MA, USA",
      volume = "33",
      number = "12",
      doi = "10.1175/1520-0469(1976)033<2452:TDEAFR>2.0.CO;2",
      pages=      "2452 - 2459",
      url = "https://journals.ametsoc.org/view/journals/atsc/33/12/1520-0469_1976_033_2452_tdeafr_2_0_co_2.xml"
}

@Article{HMMNPW2017,
AUTHOR = {Hase, N. and Miller, S. M. and Maa{\ss}, P. and Notholt, J. and Palm, M. and Warneke, T.},
TITLE = {Atmospheric inverse modeling via sparse reconstruction},
JOURNAL = {Geoscientific Model Development},
VOLUME = {10},
YEAR = {2017},
NUMBER = {10},
PAGES = {3695--3713},
URL = {https://gmd.copernicus.org/articles/10/3695/2017/},
DOI = {10.5194/gmd-10-3695-2017}
}

@article {FL1992,
      author = "Qiang  Fu and K. N.  Liou",
      title = "On the Correlated k-Distribution Method for Radiative Transfer in Nonhomogeneous Atmospheres",
      journal = "Journal of Atmospheric Sciences",
      year = "1992",
      publisher = "American Meteorological Society",
      address = "Boston MA, USA",
      volume = "49",
      number = "22",
      doi = "10.1175/1520-0469(1992)049<2139:OTCDMF>2.0.CO;2",
      pages=      "2139 - 2156",
      url = "https://journals.ametsoc.org/view/journals/atsc/49/22/1520-0469_1992_049_2139_otcdmf_2_0_co_2.xml"
}

@inproceedings{FJ1999,
  title={Computer-based underwater imaging analysis},
  author={Georges R. Fournier and Miroslaw Jonasz},
  booktitle={Optics \& Photonics},
  year={1999}
}

@article{DM2010,
	doi = {10.1088/0034-4885/73/2/026801},
	url = {https://doi.org/10.1088/0034-4885/73/2/026801},
	year = 2010,
	month = {jan},
	publisher = {{IOP} Publishing},
	volume = {73},
	number = {2},
	pages = {026801},
	author = {Anthony B Davis and Alexander Marshak},
	title = {Solar radiation transport in the cloudy atmosphere: a 3D perspective on observations and climate impacts},
	journal = {Reports on Progress in Physics},
	abstract = {The interplay of sunlight with clouds is a ubiquitous and often pleasant visual experience, but it conjures up major challenges for weather, climate, environmental science and beyond. Those engaged in the characterization of clouds (and the clear air nearby) by remote sensing methods are even more confronted. The problem comes, on the one hand, from the spatial complexity of real clouds and, on the other hand, from the dominance of multiple scattering in the radiation transport. The former ingredient contrasts sharply with the still popular representation of clouds as homogeneous plane-parallel slabs for the purposes of radiative transfer computations. In typical cloud scenes the opposite asymptotic transport regimes of diffusion and ballistic propagation coexist. We survey the three-dimensional (3D) atmospheric radiative transfer literature over the past 50 years and identify three concurrent and intertwining thrusts: first, how to assess the damage (bias) caused by 3D effects in the operational 1D radiative transfer models? Second, how to mitigate this damage? Finally, can we exploit 3D radiative transfer phenomena to innovate observation methods and technologies? We quickly realize that the smallest scale resolved computationally or observationally may be artificial but is nonetheless a key quantity that separates the 3D radiative transfer solutions into two broad and complementary classes: stochastic and deterministic. Both approaches draw on classic and contemporary statistical, mathematical and computational physics.}
}

@article{DFDM2021,
      author = "Linda Forster and Anthony B. Davis and David J. Diner and Bernhard Mayer",
      title = "Toward Cloud Tomography from Space Using MISR and MODIS: Locating the “Veiled Core” in Opaque Convective Clouds",
      journal = "Journal of the Atmospheric Sciences",
      year = "2021",
      publisher = "American Meteorological Society",
      address = "Boston MA, USA",
      volume = "78",
      number = "1",
      doi = "10.1175/JAS-D-19-0262.1",
      pages=      "155 - 166",
      url = "https://journals.ametsoc.org/view/journals/atsc/78/1/jas-d-19-0262.1.xml"
}

@article{DDET2022,
title = {Cloud tomographic retrieval algorithms. I: Surrogate minimization method},
journal = {Journal of Quantitative Spectroscopy and Radiative Transfer},
volume = {277},
pages = {107954},
year = {2022},
issn = {0022-4073},
doi = {https://doi.org/10.1016/j.jqsrt.2021.107954},
url = {https://www.sciencedirect.com/science/article/pii/S0022407321004465},
author = {Adrian Doicu and Alexandru Doicu and Dmitry Efremenko and Thomas Trautmann},
keywords = {Cloud tomographic retrieval, Multi-dimensional models},
abstract = {A cloud tomographic retrieval algorithm relying on (i) the spherical harmonics discrete ordinate method for computing the radiative transfer and (ii) the surrogate minimization method for solving the inverse problem has been designed. The retrieval algorithm uses regularization, accelerated projected gradient methods, and two types of surrogate functions. The performances of the retrieval algorithm are analyzed on a few synthetic two- and three-dimensional problems.}
}

@misc{Sta1999, 
	title={LLLab disort website}, 
	url={http://www.rtatmocn.com/disort/}, 
	journal={Light and Life Lab (LLLab)}, 
	author={Stamnes, S.}, 
	year={1999}
} 

@INPROCEEDINGS{ALHSAV2020,
  author={Aides, Amit and Levis, Aviad and Holodovsky, Vadim and Schechner, Yoav Y. and Althausen, Dietrich and Vainiger, Adi},
  booktitle={2020 IEEE International Conference on Computational Photography (ICCP)}, 
  title={Distributed Sky Imaging Radiometry and Tomography}, 
  year={2020},
  volume={},
  number={},
  pages={1-12},
  doi={10.1109/ICCP48838.2020.9105241}}

@article {MW1980,
      author = "W. E.  Meador and W. R.  Weaver",
      title = "Two-Stream Approximations to Radiative Transfer in Planetary Atmospheres: A Unified Description of Existing Methods and a New Improvement",
      journal = "Journal of Atmospheric Sciences",
      year = "1980",
      publisher = "American Meteorological Society",
      address = "Boston MA, USA",
      volume = "37",
      number = "3",
      doi = "10.1175/1520-0469(1980)037<0630:TSATRT>2.0.CO;2",
      pages=      "630 - 643",
      url = "https://journals.ametsoc.org/view/journals/atsc/37/3/1520-0469_1980_037_0630_tsatrt_2_0_co_2.xml"
}


-->

# References

1) <a id="cite-STWJ1988"/><sup><a href=#ref-1>[^]</a><a href=#ref-2>[^]</a><a href=#ref-3>[^]</a><a href=#ref-7>[^]</a><a href=#ref-9>[^]</a><a href=#ref-14>[^]</a><a href=#ref-15>[^]</a><a href=#ref-16>[^]</a><a href=#ref-18>[^]</a></sup>Knut Stamnes and S-Chee Tsay and Warren Wiscombe and Kolf Jayaweera. 1988. _Numerically stable algorithm for discrete-ordinate-method radiative transfer in multiple scattering and emitting layered media_. [URL](http://opg.optica.org/ao/abstract.cfm?URI=ao-27-12-2502)

2) <a id="cite-Sta1999"/><sup><a href=#ref-4>[^]</a><a href=#ref-6>[^]</a><a href=#ref-8>[^]</a></sup>Stamnes, S.. 1999. _LLLab disort website_. [URL](http://www.rtatmocn.com/disort/)

3) <a id="cite-SS1981"/><sup><a href=#ref-5>[^]</a></sup>Knut  Stamnes and Roy A.  Swanson. 1981. _A New Look at the Discrete Ordinate Method for Radiative Transfer Calculations in Anisotropically Scattering Atmospheres_. [URL](https://journals.ametsoc.org/view/journals/atsc/38/2/1520-0469_1981_038_0387_anlatd_2_0_co_2.xml)

4) <a id="cite-JWW1976"/><sup><a href=#ref-10>[^]</a></sup>J. H.  Joseph and W. J.  Wiscombe and J. A.  Weinman. 1976. _The Delta-Eddington Approximation for Radiative Flux Transfer_. [URL](https://journals.ametsoc.org/view/journals/atsc/33/12/1520-0469_1976_033_2452_tdeafr_2_0_co_2.xml)

5) <a id="cite-Wis1977"/><sup><a href=#ref-11>[^]</a></sup>W. J.  Wiscombe. 1977. _The Delta–M Method: Rapid Yet Accurate Radiative Flux Calculations for Strongly Asymmetric Phase Functions_. [URL](https://journals.ametsoc.org/view/journals/atsc/34/9/1520-0469_1977_034_1408_tdmrya_2_0_co_2.xml)

6) <a id="cite-MW1980"/><sup><a href=#ref-12>[^]</a><a href=#ref-13>[^]</a></sup>W. E.  Meador and W. R.  Weaver. 1980. _Two-Stream Approximations to Radiative Transfer in Planetary Atmospheres: A Unified Description of Existing Methods and a New Improvement_. [URL](https://journals.ametsoc.org/view/journals/atsc/37/3/1520-0469_1980_037_0630_tsatrt_2_0_co_2.xml)

7) <a id="cite-Syk1951"/><sup><a href=#ref-17>[^]</a></sup>Sykes, J. B.. 1951. _Approximate Integration of the Equation of Transfer_. [URL](https://doi.org/10.1093/mnras/111.4.377)

